In [1]:
#
# Copyright © 2019 Sunho Kim. All rights reserved.
#

In [2]:
cd ..

/gorani/gorani/backend


In [3]:
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession\
    .builder\
    .appName('App Name')\
    .getOrCreate()
sc = spark.sparkContext

In [4]:
# parameters


In [5]:
# Parameters
alpha = 0.6
ratio = 0.1


In [6]:
from gorani.spark import read_api_all, read_data_all, write_api

cb_df = read_data_all(spark, 'completed_books')

post_df = read_api_all(spark, 'sentence_posts').select('book_id', 'id')

fp_df = read_api_all(spark, 'feed_posts')

In [7]:
import pyspark.sql.functions as F
from gorani.utils import uuid

df = cb_df.join(post_df.alias('p'), cb_df['book_id'] == post_df['book_id'], 'inner')\
    .select(F.col('id').alias('post_id'), 'p.book_id', 'user_id')
df = df.join(fp_df, (df['post_id'] == fp_df['post_id']) & (df['user_id'] == fp_df['user_id']), 'left_anti')\
        .select('user_id', 'post_id')\
        .withColumn('id', uuid())\
        .withColumn('updated_at', F.current_timestamp())\
        .withColumn('created_at', F.current_timestamp())

write_api('feed_posts', df)